In [1]:
#Interlinking 
import urllib.request
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
from SPARQLWrapper import SPARQLWrapper, JSON

 

#Definizione funzione per creazione URI 
def urify(ns, testo):
    testo=testo.replace(" ","_").replace(".","")
    return ns+urllib.parse.quote(testo)


    
def existsInDBPedia(dictionary,resource_name):
    #serve solamente per avere il formato desiderato
    formatted_resource_name = urify("", resource_name).title().replace(" ", "_")
    dbpedia_asked_uri = urify("http://dbpedia.org/resource/",formatted_resource_name)
    #abbiamo già trovato questa risorsa su dbPedia?
    resource = dictionary.get(formatted_resource_name)
    if (resource != None):
        return resource
    else: 
        sparql.setQuery("ASK { <http://dbpedia.org/resource/"  + formatted_resource_name + "> ?p ?o } ")
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        dictionary[formatted_resource_name] = results["boolean"]
        return results["boolean"] #restituisce "True" o "False"



 

g = Graph()
rso = Namespace("http://www.regionesiciliana.it/ontology/")
base_uri="http://www.regionesiciliana.it/resource/"
g.bind("rso", rso)
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

 


#Apertura file csv con il nostro dataset
with open('mercatiStoriciSiciliani.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    #Lettura file e definizioe specifiche URI
    dictionary = {}
    for row in reader:
        #Creazione URI necessarie
        uri_HistoricalMarket=urify(base_uri , str(row['ObjectID'])) 
        uri_GeoPoint= urify(base_uri, "coordinates" + str(row['ObjectID']))
        
        #Impostazione localizzazione 
        g.add([URIRef(uri_HistoricalMarket), rso.localization, Literal(row['Localizzazione'])])
        #Impostazione denominazione del mercato
        g.add([URIRef(uri_HistoricalMarket), rso.name, Literal(row['Denominazione'])])
        #Impostazione coordinate geografiche di ogni mercato
        g.add([URIRef(uri_HistoricalMarket) , rso.isLocated, URIRef(uri_GeoPoint)])
        g.add([URIRef(uri_GeoPoint), rso.latitude, Literal(row['Latitudine'])])
        g.add([URIRef(uri_GeoPoint), rso.longitude, Literal(row['Longitudine'])])
        #Impostazione descrizione di ciascun. mercato
        g.add([URIRef(uri_HistoricalMarket), rso.description, Literal(row['Descrizione'], lang='it' )])
        #Impostazione tipologia di ciascun mercato
            #verifichiamo se esite la risorsa per la tipologia su dbpedia
        type_boolean = existsInDBPedia(dictionary, str(row["Tipologia"]))
        if (type_boolean):
            formatted_resource_name = urify("",str(row["Tipologia"])).title().replace(" ", "_")
            dbpedia_asked_uri = urify("http://dbpedia.org/resource/",formatted_resource_name)
            print(uri_HistoricalMarket + " rso:type " + "dbpedia_asked_uri")
            g.add([URIRef(uri_HistoricalMarket) , rso.type, URIRef(dbpedia_asked_uri)])
        else:
            #non esiste su dbpedia, la creiamo noi
            print(uri_HistoricalMarket + " rso:type " + Literal(row['Tipologia']))
            g.add([URIRef(uri_HistoricalMarket) , rso.type, Literal(row["Tipologia"])])

        #Impostazione Comune dove è situato ciascun mercato
        #verifichiamo se esite la risorsa per la town su dbpedia
        town_boolean = existsInDBPedia(dictionary, str(row["Comune"]))
        if (town_boolean):
            formatted_resource_name = urify("",str(row["Comune"])).title().replace(" ", "_")
            dbpedia_asked_uri = urify("http://dbpedia.org/resource/",formatted_resource_name)
            print(uri_HistoricalMarket + " rso:town " + dbpedia_asked_uri)
            g.add([URIRef(uri_HistoricalMarket) , rso.town, URIRef(dbpedia_asked_uri)])
        else:
            #non esiste su dbpedia, la creiamo noi
            print(uri_HistoricalMarket + " rso:town " + Literal(row["Comune"]))
            g.add([URIRef(uri_HistoricalMarket) , rso.town, Literal(row["Comune"])])
            
        #Impostazione Provincia 
            #verifichiamo se esite la risorsa per la provincia su dbpedia
        province_boolean = existsInDBPedia(dictionary, str(row["Provincia"]))
        if (province_boolean):
            formatted_resource_name = urify("",str(row["Provincia"])).title().replace(" ", "_")
            dbpedia_asked_uri = urify("http://dbpedia.org/resource/",formatted_resource_name)
            print(uri_HistoricalMarket + " rso:province " + dbpedia_asked_uri)
            g.add([URIRef(uri_HistoricalMarket) , rso.province, URIRef(dbpedia_asked_uri)])
        else:
            #non esiste su dbpedia, la creiamo noi
            print(uri_HistoricalMarket + " rso:province " + Literal(row["Provincia"]))
            g.add([URIRef(uri_HistoricalMarket) , rso.province, Literal(row["Provincia"])])
            
            
        #Controlliamo che esista la risorsa mercato storico su dbpedia 
        boolean = existsInDBPedia(dictionary, str(row["Denominazione"]))
        dbpedia_asked_uri = urify("http://dbpedia.org/resource/",str(row["Denominazione"]))
        if(boolean == True):
            print(uri_HistoricalMarket + " owl:sameAs " + dbpedia_asked_uri)
            g.add([URIRef(uri_HistoricalMarket), OWL.sameAs, URIRef(dbpedia_asked_uri)])
            
 
    g.serialize(destination='ontologiaGenerata.ttl', format='turtle')      
       
    f1=open('definizioneOntologia.ttl', 'a+')
    f2=open('FinalOntology.ttl', 'r')
    f1.write(f2.read())
    f1.close()
    f2.close()
            
        

http://www.regionesiciliana.it/resource/1 rso:type Mercato Ittico
http://www.regionesiciliana.it/resource/1 rso:town http://dbpedia.org/resource/Sciacca
http://www.regionesiciliana.it/resource/1 rso:province http://dbpedia.org/resource/Agrigento
http://www.regionesiciliana.it/resource/1 owl:sameAs http://dbpedia.org/resource/Mercato
http://www.regionesiciliana.it/resource/2 rso:type Mercato Ittico
http://www.regionesiciliana.it/resource/2 rso:town http://dbpedia.org/resource/Porto_Empedocle
http://www.regionesiciliana.it/resource/2 rso:province http://dbpedia.org/resource/Agrigento
http://www.regionesiciliana.it/resource/2 owl:sameAs http://dbpedia.org/resource/Mercato
http://www.regionesiciliana.it/resource/3 rso:type Mercato Ittico
http://www.regionesiciliana.it/resource/3 rso:town http://dbpedia.org/resource/Licata
http://www.regionesiciliana.it/resource/3 rso:province http://dbpedia.org/resource/Agrigento
http://www.regionesiciliana.it/resource/3 owl:sameAs http://dbpedia.org/resou

http://www.regionesiciliana.it/resource/31 rso:town http://dbpedia.org/resource/Scoglitti
http://www.regionesiciliana.it/resource/31 rso:province http://dbpedia.org/resource/Ragusa
http://www.regionesiciliana.it/resource/31 owl:sameAs http://dbpedia.org/resource/Mercato
http://www.regionesiciliana.it/resource/32 rso:type Mercato Ittico
http://www.regionesiciliana.it/resource/32 rso:town http://dbpedia.org/resource/Lampedusa
http://www.regionesiciliana.it/resource/32 rso:province http://dbpedia.org/resource/Agrigento
http://www.regionesiciliana.it/resource/32 owl:sameAs http://dbpedia.org/resource/Mercato
http://www.regionesiciliana.it/resource/33 rso:type Mercato Ittico
http://www.regionesiciliana.it/resource/33 rso:town http://dbpedia.org/resource/Portopalo
http://www.regionesiciliana.it/resource/33 rso:province http://dbpedia.org/resource/Menfi
http://www.regionesiciliana.it/resource/33 owl:sameAs http://dbpedia.org/resource/Mercato
http://www.regionesiciliana.it/resource/34 rso:type